In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import requests
from datetime import datetime

Initialize a DataFrame to store web scraped data.

In [2]:
re_df = pd.DataFrame().assign(Raw_Price=[], Raw_Beds=[], Raw_Baths=[], 
                     Raw_Square_Feet=[], Raw_Year_Built=[], Raw_Garages=[], 
                     Raw_Lot_SqFt=[], Raw_Date_Sold=[], Raw_Days_On_Market=[])

## Web Scraping
Scrape MLS data from [mlslistings.com](https://www.mlslistings.com). Loop through all pages of properties that have sold in Folsom in the past year, collecting on different house characteristics.

In [ ]:
i = 1
while i:
    url = f'https://www.mlslistings.com/Search/Result/8ca51c7a-f3b6-4f02-b070-1d0abf2168f7/{i}'
    page = requests.get(url)
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get(url)
    soup = BeautifulSoup(driver.page_source)
    try:
        soup.find('strong', {'class': "listing-price d-block pull-left pr-25"}).text
    except:
        break
    price = [x.text for x in soup.find_all('strong', {'class': "listing-price d-block pull-left pr-25"})]
    baths = [x.text for x in soup.find_all('span', {'class': "listing-info-item font-size-sm line-height-base d-block pull-left pr-50 listing-baths"})]
    beds = [x.text for x in soup.find_all('span', {'class': "listing-info-item font-size-sm line-height-base d-block pull-left pr-50 listing-beds"})]
    sqft_yb = [x.text for x in soup.find_all('span', {'class': "listing-info-item font-size-sm line-height-base d-block pull-left pr-50 listing-sqft last"})]
    sqft = [sqft_yb[i] for i in range(len(sqft_yb)) if i % 2 == 0]
    yb = [sqft_yb[i] for i in range(len(sqft_yb)) if i % 2 == 1]
    garages = [x.text for x in soup.find_all('span', {'class': "listing-info-item font-size-sm line-height-base d-block pull-left pr-50 listing-garage"})]
    lot = [x.text for x in soup.find_all('span', {'class': "listing-info-item font-size-sm line-height-base d-block pull-left pr-50 listing-lot-size"})]
    date_sold = [x.text for x in soup.find_all('span', {'class': "listing-statusd-block pull-left pl-50 pr-1 status-marker status-closed"})]
    days_active = [x.text for x in soup.find_all('span', {'class': "listing-dom-block pull-left pl-25 hidden"})]
    new_df = pd.DataFrame().assign(Raw_Price=price, Raw_Beds=beds, Raw_Baths=baths, 
                 Raw_Square_Feet=sqft, Raw_Year_Built=yb, Raw_Garages=garages, 
                 Raw_Lot_SqFt=lot, Raw_Date_Sold=date_sold, Raw_Days_On_Market=days_active)
    re_df = pd.concat([re_df, new_df])
    i += 1 

In [ ]:
re_df

## Data Cleaning
Remove unnecessary non-number characters from data set, convert number strings into ints and floats, convert measurements in acres to square feet, change data sold into numeric date format.

In [ ]:
re_df['Price'] = [int(x.replace('$','').replace(',','')) for x in re_df['Price']]
re_df['Baths'] = [float(x.replace('\n','').replace('/','.').replace('Ba', '')) for x in re_df['Baths']]
re_df['Beds'] = [int(x.replace('\n','').replace('Bd','')) for x in re_df['Beds']]
re_df['Square_Feet'] = [int(x.replace('\n','').replace('Sq Ft','').replace(',','')) for x in 
                        re_df['Square_Feet']]
re_df['Year_Built'] = [int(x.replace('\n', '').replace('Year Built', '')) for x in re_df['Year_Built']]
re_df['Garages'] = [int(x.replace('\n', '').replace('Garage', '')) for x in re_df['Garages']]
re_df['Lot_SqFt'] = [int(x.replace('\n','').replace('Sq Ft Lot','').replace(',','').replace('–','0')) if 
                     'Sq Ft' in x else int(float(x.replace('\n','').replace('Acres Lot','').replace(',','')
                                                 .replace('–','0')) * 45360) for x in re_df['Lot_SqFt']]
re_df['Date_Sold'] = [f'{months_dict[x[0]]}/{x[1]}/{x[2]}' if len(x[1]) == 2 else 
                      f'{months_dict[x[0]]}/0{x[1]}/{x[2]}'for x in [y.replace('Sold on ','').replace(',','')
                                                                     .split() for y in re_df['Date_Sold']]]
re_df['Days_On_Market'] = [int(x.strip(' Days on Site')) for x in re_df['Days_On_Market']]
re_df = re_df.reset_index()
# Many brand new homes do not add a year built to the MLS data, which results the year built being recorded as 0.
# To fix this, the year built of homes with this problem is changed to the year the house sold.
re_df['Year_Built'] = [int(re_df['Date_Sold'][i][-4:]) if re_df['Year_Built'][i] == 0 else 
                       re_df['Year_Built'][i] for i in range(re_df.shape[0])]

Data now looks ready to be analyzed.

In [57]:
re_df

,index,Price,Baths,Beds,Square_Feet,Year_Built,Garages,Lot_SqFt,Date_Sold,Days_On_Market
0,0,800000,3.0,4,2350,1999,3,7518,08/28/2023,47
1,1,840000,3.0,4,2319,1996,3,8695,08/28/2023,26
2,2,685000,2.0,4,2002,1989,3,7932,08/28/2023,25
3,3,575000,2.0,3,1198,1993,2,5144,08/28/2023,25
4,4,840000,3.0,3,2306,1989,3,14651,08/28/2023,33
...,...,...,...,...,...,...,...,...,...,...
725,12,1250000,3.1,5,4216,2006,8,12111,08/30/2022,441
726,13,1125000,2.1,4,3102,1983,3,18370,08/30/2022,417
727,14,772000,3.0,4,2335,2019,2,4996,08/30/2022,476
728,15,558000,2.0,2,1245,1999,2,4356,08/30/2022,389


In [ ]:
re_df.to_csv(f'mlssales{datetime.now().strftime("%y-%m-%d")}.csv')